In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
net=cv2.dnn.readNetFromDarknet("yolov3_custom.cfg","yolov3_custom_10000.weights")

In [3]:
classes=['Arla sour cream','Arla ecological sour cream','Arla sour milk','Alpro blueberry soyghurt','Alpro vanilla soyghurt',
         'Alpro fresh soymilk','Arla mild vanilla yoghurt','Arla natural mild low fat yoghurt','Arla natural yoghurt',
         'Valio vanilla yoghurt','Yoggi strawberry yoghurt','Yoggi vanilla yoghurt','Bravo-Apple-Juice','Bravo-Orange-Juice',
         'God-Morgon-Apple-Juice','God-Morgon-Orange-Juice','God-Morgon-Orange-Red-Grapefruit-Juice','God-Morgon-Red-Grapefruit-Juice',
         'Tropicana-Apple-Juice','Tropicana-Golden-Grapefruit','Tropicana-Juice-Smooth','Tropicana-Mandarin-Morning',
         'Arla-Ecological-Medium-Fat-Milk','Arla-Lactose-Medium-Fat-Milk','Arla-Medium-Fat-Milk','Arla-Standard-Milk',
         'Garant-Ecological-Medium-Fat-Milk','Garant-Ecological-Standard-Milk','Oatly-Oat-Milk','Oatly-Natural-Oatghurt',
         'Red-Kideny-Beans','White-Beans','Green-Beans','Cake','Muffin','Water','Cocacola','Pepsi']

In [4]:
net.getUnconnectedOutLayers()
#array([[200], [227], [254]])

array([200, 227, 254])

In [5]:
layer_names = net.getLayerNames()
layer_names=list(layer_names)
outputlayers = [layer_names[i[0] - 1] for i in ([[200], [227], [254]])]
outputlayers
#layer_names

['yolo_82', 'yolo_94', 'yolo_106']

In [6]:
colors= np.random.uniform(0,255,size=(len(classes),3))

## Generate a method to detect videos

In [7]:
import time
def detect_videos():
    #cap=cv2.VideoCapture(video_path)
    #For Webcam
    cap = cv2.VideoCapture(0)
    font = cv2.FONT_HERSHEY_PLAIN
    starting_time= time.time()
    frame_id = 0
    
    while True:
        ret,frame= cap.read()
        frame_id+=1
        height,width,channels = frame.shape
        #Detecting objects
        blob = cv2.dnn.blobFromImage(frame,0.00392,(320,320),(0,0,0),True,crop=False) #reduce 416 to 320 
        net.setInput(blob)
        outs = net.forward(outputlayers)
        #print(outs[1])
        
        #Showing info on screen/ get confidence score of algorithm in detecting an object in blob
        class_ids=[]
        confidences=[]
        boxes=[]
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                
                if confidence > 0.3:
                    #onject detected
                    center_x= int(detection[0]*width)
                    center_y= int(detection[1]*height)
                    w = int(detection[2]*width)
                    h = int(detection[3]*height)
                    
                    #cv2.circle(img,(center_x,center_y),10,(0,255,0),2)
                    #rectangle co-ordinaters
                    x=int(center_x - w/2)
                    y=int(center_y - h/2)
                    #cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

                    boxes.append([x,y,w,h]) #put all rectangle areas
                    confidences.append(float(confidence)) #how confidence was that object detected and show that percentage
                    class_ids.append(class_id) #name of the object tha was detected
            
        indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)
        
        for i in range(len(boxes)):
            if i in indexes:
                x,y,w,h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence= confidences[i]
                #color = colors[class_ids[i]]
                color = (0,0,255)
                cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
                cv2.putText(frame,label+" "+str(round(confidence,2)),(x,y+30),font,1,(255,255,255),2)
            
        elapsed_time = time.time() - starting_time
        fps=frame_id/elapsed_time
        cv2.putText(frame,"FPS:"+str(round(fps,2)),(10,50),font,2,(0,0,255),1)
    
        cv2.imshow("Image",frame)
        key = cv2.waitKey(1) #wait 1ms the loop will start again and we will process the next frame
    
        if key == 27: #esc key stops the process
            break;        
    cap.release()    
    cv2.destroyAllWindows() 
    return class_ids

In [8]:
detect_videos()

[]